<a href="https://colab.research.google.com/github/Barbarioli/Pipeline/blob/master/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pipeline

##Packages

### Multiprocessing

In [0]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import cv2
from multiprocessing import Process, JoinableQueue, Pool, Manager, cpu_count
from time import sleep
import time
import sys
import os
import sys
import pathlib
import logging
import traceback 
import socket
import time
import shutil
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import \
    ReduceLROnPlateau as ReduceLROnPlateauPyTorch
from torch.optim.lr_scheduler import MultiStepLR

In [4]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/Photos"

####Loading modules

In [0]:
#Changing path to folder
sys.path.append("/content/drive/My Drive/bandlimited-cnns-master/cnns/nnlib/utils")

from general_utils import OptimizerType
from general_utils import SchedulerType
from general_utils import LossType
from general_utils import LossReduction
from general_utils import TensorType
from general_utils import PrecisionType
from general_utils import additional_log_file
from general_utils import mem_log_file
from general_utils import get_log_time

In [0]:
#Changing path to folder
sys.path.append("/content/drive/My Drive/bandlimited-cnns-master/cnns/nnlib/pytorch_layers")

from AdamFloat16 import AdamFloat16

In [18]:
#Changing path to folder
#sys.path.append("/content/drive/My Drive/bandlimited-cnns-master/cnns/nnlib/datasets")
#from mnist import get_mnist
#from cifar import get_cifar
sys.path.append("/content/drive/My Drive/bandlimited-cnns-master/cnns/nnlib/datasets/ucr")
from ucr import get_ucr

ImportError: ignored

In [8]:
# from cnns.nnlib.pytorch_architecture.model_utils import getModelPyTorch

from cnns.nnlib.datasets.mnist import get_mnist
from cnns.nnlib.datasets.cifar import get_cifar
from cnns.nnlib.datasets.ucr.ucr import get_ucr
from cnns.nnlib.utils.exec_args import get_args
from cnns.nnlib.pytorch_experiments.utils.progress_bar import progress_bar
from cnns.nnlib.pytorch_architecture.le_net import LeNet
from cnns.nnlib.pytorch_architecture.resnet2d import resnet18
from cnns.nnlib.pytorch_architecture.densenet import densenet_cifar
from cnns.nnlib.pytorch_architecture.fcnn import FCNNPytorch
from cnns.nnlib.utils.general_utils import NetworkType

"""
sources:
https://devblogs.nvidia.com/apex-pytorch-easy-mixed-precision-training/
"""
try:
    import apex
except ImportError:
    raise ImportError("""Please install apex from 
    https://www.github.com/nvidia/apex to run this code.""")

amp_handle = None
# from apex.parallel import DistributedDataParallel as DDP
# from apex.fp16_utils import input_to_half
from apex.fp16_utils import network_to_half
from apex.fp16_utils import FP16_Optimizer


ModuleNotFoundError: ignored

##Loading

In [0]:
def load(path):
  """
  path: path to the photos to be loaded
  """
  image_files = sorted([os.path.join(path, file) for file in os.listdir(path) if file.endswith('.jpg')])
  return image_files

##Inference

In [3]:
models_folder_name = "models"
models_dir = os.path.join(os.getcwd(), models_folder_name)
print("models_dir: ", models_dir)
pathlib.Path(models_dir).mkdir(parents=True, exist_ok=True)
# if not os.path.exists(models_dir):
#     os.makedirs(models_dir)

# plt.switch_backend('agg')

args = get_args()

current_file_name = __file__.split("/")[-1].split(".")[0]
print("current file name: ", current_file_name)

if torch.cuda.is_available() and args.use_cuda:
    print("cuda is available: ")
    device = torch.device("cuda")
    # torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    device = torch.device("cpu")


def getModelPyTorch(args):
    """
    Get the PyTorch version of the FCNN model.
    :param input_size: the length (width) of the time series.
    :param num_classes: number of output classes.
    :param in_channels: number of channels in the input data for a convolution.
    :param out_channels: number of channels in the output of a convolution.
    :param dtype: global - the type of torch data/weights.
    :param flat_size: the size of the flat vector after the conv layers.
    :return: the model.
    """
    network_type = args.network_type
    if network_type is NetworkType.LE_NET:
        return LeNet(args=args)
    elif network_type is NetworkType.FCNN_SMALL or (
            network_type is NetworkType.FCNN_STANDARD):
        if network_type is NetworkType.FCNN_SMALL:
            args.out_channels = [1, 1, 1]
        elif network_type is NetworkType.FCNN_STANDARD:
            args.out_channels = [128, 256, 128]
        return FCNNPytorch(args=args)
    elif network_type == NetworkType.ResNet18:
        return resnet18(args=args)
    elif network_type == NetworkType.DenseNetCifar:
        return densenet_cifar(args=args)
    else:
        raise Exception("Unknown network_type: ", network_type)


def readucr(filename, data_type):
    parent_path = os.path.split(os.path.abspath(dir_path))[0]
    print("parent path: ", parent_path)
    filepath = os.path.join(parent_path, ucr_data_folder, filename,
                            filename + "_" + data_type)
    print("filepath: ", filepath)
    data = np.loadtxt(filepath, delimiter=',')
    Y = data[:, 0]
    X = data[:, 1:]
    return X, Y


def getData(fname):
    x_train, y_train = readucr(fname + '/' + fname + '_TRAIN')
    x_test, y_test = readucr(fname + '/' + fname + '_TEST')
    num_classes = len(np.unique(y_test))
    batch_size = min(x_train.shape[0] // 10, args.min_batch_size)

    y_train = ((y_train - y_train.min()) / (y_train.max() - y_train.min()) * (
            num_classes - 1)).astype(int)
    y_test = ((y_test - y_test.min()) / (y_test.max() - y_test.min()) * (
            num_classes - 1)).astype(int)

    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean) / x_train_std

    x_test = (x_test - x_train_mean) / x_train_std
    # Add a single channels at the end of the data.
    x_train = x_train.reshape(x_train.shape + (1,))
    x_test = x_test.reshape(x_test.shape + (1,))

    return x_train, y_train, x_test, y_test, batch_size, num_classes


# @profile


def test(model, device, test_loader, loss_function, args, epoch=None):
    """
    Test the model and return test loss and accuracy.

    :param model: deep learning model.
    :param device: cpu or gpu.
    :param test_loader: the input data.
    :param dataset_type: test or train.
    :param dtype: the data type of the tensor.
    :param epoch: current epoch of the model training/testing.
    :return: test loss and accuracy.
    """
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    #Profile inference time and memory
    inference_time = 0
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    memory_tensor = 0
    memory_cache = 0

    with torch.no_grad():
        torch.cuda.empty_cache()
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device=device, dtype=args.dtype), target.to(
                device)
            start.record()
            output = model(data)
            end.record()
            memory_tensor += torch.cuda.memory_allocated(device=None)
            memory_cache += torch.cuda.memory_cached(device=None)
            
            torch.cuda.synchronize()
            inference_time += start.elapsed_time(end)
            test_loss += loss_function(output,
                                       target).item()  # sum up batch loss
            # get the index of the max log-probability
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

            if args.log_conv_size is True:
                with open(additional_log_file, "a") as file:
                    file.write("\n")

            if args.is_progress_bar:
                progress_bar(total, len(test_loader.dataset), epoch=epoch,
                             msg="Test Loss: %.3f | Test Acc: %.3f%% (%d/%d)" %
                                 (test_loss / total, 100. * correct / total,
                                  correct,
                                  total))

        # Test loss for the whole dataset.
        inference_time = inference_time / total
        test_loss /= total
        accuracy = 100. * correct / total
        memory_cache = int(memory_cache / len(test_loader))
        memory_tensor = int(memory_tensor / len(test_loader))
         
        with open("memory_time_profile", "a") as file:        
            file.write("-compress-rate- " + str(args.compress_rate) + "accuracy " + str(accuracy) + " inference_time_milliseconds " \
                + str(inference_time) + " memory_tensor_mb: {}".format(memory_tensor >> 20) + \
                " memory_cache: {}".format(memory_cache >> 20) + "\n")
        

        return test_loss, accuracy

# @profile
def main(args):
    """
    The main training.

    :param dataset_name: the name of the dataset from UCR.
    """
    is_debug = args.is_debug
    dataset_name = args.dataset_name
    preserve_energy = args.preserve_energy
    compress_rate = args.compress_rate

    torch.manual_seed(args.seed)
    optimizer_type = args.optimizer_type
    scheduler_type = args.scheduler_type
    loss_type = args.loss_type
    loss_reduction = args.loss_reduction

    use_cuda = args.use_cuda
    device = torch.device("cuda" if use_cuda else "cpu")
    tensor_type = args.tensor_type
    if use_cuda and args.noise_sigma is False:
        if tensor_type is TensorType.FLOAT32:
            cuda_type = torch.cuda.FloatTensor
        elif tensor_type is TensorType.FLOAT16:
            cuda_type = torch.cuda.HalfTensor
        elif tensor_type is TensorType.DOUBLE:
            cuda_type = torch.cuda.DoubleTensor
        else:
            raise Exception(f"Unknown tensor type: {tensor_type}")
        # The below has to be disabled for normal distribution to work (add noise).
        torch.set_default_tensor_type(cuda_type)
    elif use_cuda is False and args.noise_sigma is False:
        if tensor_type is TensorType.FLOAT32:
            cpu_type = torch.FloatTensor
        elif tensor_type is TensorType.FLOAT16:
            cpu_type = torch.HalfTensor
        elif tensor_type is TensorType.DOUBLE:
            cpu_type = torch.DoubleTensor
        else:
            raise Exception(f"Unknown tensor type: {tensor_type}")
        torch.set_default_tensor_type(cpu_type)

    train_loader, dev_loader, test_loader = None, None, None

    model = getModelPyTorch(args=args)
    model.to(device)
    # model = torch.nn.DataParallel(model)

    # https://pytorch.org/docs/master/notes/serialization.html
    if args.model_path != "no_model":
        model.load_state_dict(
            torch.load(os.path.join(models_dir, args.model_path),
                       map_location=device))
        msg = "loaded model: " + args.model_path
        # logger.info(msg)
        print(msg)
    if args.precision_type is PrecisionType.FP16:
        # model.half()  # convert to half precision
        model = network_to_half(model)
    """
    You want to make sure that the BatchNormalization layers use float32 for 
    accumulation or you will have convergence issues.
    https://discuss.pytorch.org/t/training-with-half-precision/11815
    """
    # for layer in model.modules():
    #     if isinstance(layer, nn.BatchNorm1d) or isinstance(layer,
    #                                                        nn.BatchNorm2d):
    #         layer.float()

    params = model.parameters()
    eps = 1e-8

    if optimizer_type is OptimizerType.MOMENTUM:
        optimizer = optim.SGD(params, lr=args.learning_rate,
                              momentum=args.momentum,
                              weight_decay=args.weight_decay)
    elif optimizer_type is OptimizerType.ADAM_FLOAT16:
        optimizer = AdamFloat16(params, lr=args.learning_rate, eps=eps)
    elif optimizer_type is OptimizerType.ADAM:
        optimizer = optim.Adam(params, lr=args.learning_rate,
                               betas=(args.adam_beta1, args.adam_beta2),
                               weight_decay=args.weight_decay, eps=eps)
    else:
        raise Exception(f"Unknown optimizer type: {optimizer_type.name}")

    # https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.ReduceLROnPlateau
    if scheduler_type is SchedulerType.ReduceLROnPlateau:
        scheduler = ReduceLROnPlateauPyTorch(optimizer=optimizer, mode='min',
                                             factor=0.1, patience=10)
    elif scheduler_type is SchedulerType.MultiStepLR:
        scheduler = MultiStepLR(optimizer=optimizer, milestones=[150, 250])
    else:
        raise Exception(f"Unknown scheduler type: {scheduler_type}")

    if args.precision_type is PrecisionType.FP16:
        """
        amp_handle: tells it where back-propagation occurs so that it can 
        properly scale the loss and clear internal per-iteration state.
        """
        # amp_handle = amp.init()
        # optimizer = amp_handle.wrap_optimizer(optimizer)

        # The optimizer supported by apex.
        optimizer = FP16_Optimizer(optimizer,
                                   static_loss_scale=args.static_loss_scale,
                                   dynamic_loss_scale=args.dynamic_loss_scale,
                                   verbose=True)

    # Optionally resume from a checkpoint.
    if args.resume:
        # Use a local scope to avoid dangling references.
        def resume():
            if os.path.isfile(args.resume):
                print("=> loading checkpoint '{}'".format(args.resume))
                checkpoint = torch.load(args.resume,
                                        map_location=lambda storage,
                                                            loc: storage.cuda(
                                            args.gpu))
                args.start_epoch = checkpoint['epoch']
                max_train_accuracy = checkpoint['max_train_accuracy']
                model.load_state_dict(checkpoint['state_dict'])
                # An FP16_Optimizer instance's state dict internally stashes the master params.
                optimizer.load_state_dict(checkpoint['optimizer'])
                print("=> loaded checkpoint '{}' (epoch {})"
                      .format(args.resume, checkpoint['epoch']))
                return max_train_accuracy
            else:
                print("=> no checkpoint found at '{}'".format(args.resume))
                return 0.0

        max_train_accuracy = resume()

    if loss_reduction is LossReduction.ELEMENTWISE_MEAN:
        reduction_function = "mean"
    elif loss_reduction is LossReduction.SUM:
        reduction_function = "sum"
    else:
        raise Exception(f"Unknown loss reduction: {loss_reduction}")

    if loss_type is LossType.CROSS_ENTROPY:
        loss_function = torch.nn.CrossEntropyLoss(reduction=reduction_function)
    elif loss_type is LossType.NLL:
        loss_function = torch.nn.NLLLoss(reduction=reduction_function)
    else:
        raise Exception(f"Unknown loss type: {loss_type}")

    test_time = time.time()
    test(model = model, device = device, test_loader = test_loader, loss_function = loss_function, args = args)
    print("test time", time.time() - test_time)
  
    start_time = time.time()
    hostname = socket.gethostname()
    cuda_visible_devices = os.environ['CUDA_VISIBLE_DEVICES']
    global_log_file = os.path.join(results_folder_name,
                                   get_log_time() + "-ucr-fcnn.log")
    args_str = args.get_str()
    HEADER = "hostname," + str(
        hostname) + ",timestamp," + get_log_time() + "," + str(
        args_str) + ",cuda_visible_devices," + str(cuda_visible_devices)

    if args.precision_type is PrecisionType.AMP:
        from apex import amp

        amp_handle = amp.init(enabled=True)

    if args.dataset == "all" or args.dataset == "ucr":
        flist = os.listdir(ucr_path)
    elif args.dataset == "cifar10":
        flist = ["cifar10"]
    elif args.dataset == "cifar100":
        flist = ["cifar100"]
    elif args.dataset == "mnist":
        flist = ["mnist"]

    else:
        raise AttributeError("Unknown dataset: ", args.dataset)

    if torch.cuda.is_available():
        print("CUDA is available")
        torch.cuda.empty_cache()
    else:
        print("CUDA is not available")

    # flist = sorted(flist, key=lambda s: s.lower())
    # flist = flist[3:]  # start from Beef
    # reversed(flist)
    print("flist: ", flist)
    for dataset_name in flist:
        args.dataset_name = dataset_name
        print("Dataset: ", dataset_name)
        for preserve_energy in args.preserve_energies:
            print("preserve energy: ", preserve_energy)
            args.preserve_energy = preserve_energy
            for compress_rate in args.compress_rates:
                print("compress rate: ", compress_rate)
                args.compress_rate = compress_rate
                for noise_sigma in args.noise_sigmas:
                    print("noise sigma: ", noise_sigma)
                    args.noise_sigma = noise_sigma
                    start_training = time.time()
                    try:
                        main(args=args)
                    except RuntimeError as err:
                        print(f"ERROR: {dataset_name}. "
                              "Details: " + str(err))
                        traceback.print_tb(err.__traceback__)
                    print("elapsed time (sec): ", time.time() - start_training)

    print("total elapsed time (sec): ", time.time() - start_time)


models_dir:  /content/models


NameError: ignored

##Operators

In [0]:
class image_loader(object):
  """Load images from a list of paths to a queue in order to be processed.
  
  Args:
    data: list of paths to all images.
    queue: queue to where the loaded images will be added.
  
  """
  
  def __init__(self, data, queue):
    self.queue = queue
    self.data = iter(data)
    
  def __iter__(self):  
    return self
  
  def __next__(self):
    if len(data) > 0:
      self.queue.put(imread(data[-1]))
      data.pop()
      return
    
    else:
      raise StopIteration
      
      
class preprocessing(object):
  """Preprocess images resizing, cropping and converting to tensor
  
  Args:
    input_queue: input queue with the images to be preprocessed.
    output_queue: queue with the output images.
    output_size (tuple): desired image size in a tuple of (height, width).
    crop (boolean): randomly crop the image. If cropped it will not be resized.
    totensor (boolean): if the images should be converted to tensor.
    
  """
  
  def __init__(self, input_queue, output_queue, output_size, crop = False, totensor = False):
    self.input_queue = input_queue
    self.output_queue = output_queue
    self.size = output_size
    self.crop = crop
    self.tensor = totensor
    
  def __iter__(self):
    return self
      
  def __next__(self):
    if self.input_queue.qsize() != 0:
      
      image = self.input_queue.get()
      
      if self.crop:
        h, w = image.shape[:2]
        new_h, new_w = self.size
        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)
        image = image[top: top + new_h,
                     left: left + new_w]
        
      else:
        image = resize(image, self.size, anti_aliasing = True)
      
      if self.tensor:
        image = image.transpose((2,0,1))
        image = torch.from_numpy(image)
        
      self.output_queue.put(image)
      
      
      
class inference(object):
  """Perform inference on a dataset
  
  Args:
    input_queue: input queue with the images to be preprocessed.
    output_queue: queue with the output images.
    compression_rate(float): float number between 0-100 which represents the compression rate
          in percentage terms. For simulation it represents the sleep time in seconds
    simulation(boolean): if simulation or real run
    model(string): the deep learning model that will be used on the images
    
  """
  
  def __init__(self, input_queue, output_queue, compression_rate, simulation = True, model = None):
      self.input_queue = input_queue
      self.output_queue = output_queue
      self.simulation = simulation
      
      if simulation:
        self.sleep = compression_rate
        
      else:
        self.compression_rate = compression_rate
        self.model = model
      
  def __iter__(self):
      return self
  
  def __next__(self):
    if self.simulation:
      self.input_queue.get()
      sleep(self.sleep)
      self.output_queue.put('a')
      return
    #else:
       

##Test

In [0]:
data = load("/content/drive/My Drive/Photos")

In [0]:
#Initializing queues

queue_1 = JoinableQueue()
queue_2 = JoinableQueue()
queue_3 = JoinableQueue()

#Loading

process_1 = Process(target = image_loader, args = (data, queue_1), daemon=True)
process_1.start()

#Preprocessing

process_2 = Process(target = preprocessing, args = (queue_1, queue_2, (300,300)), daemon=True)
process_2.start()

#Inference

process_3 = Process(target = inference, args = (queue_2, queue_3, 2), daemon=True)
process_3.start()

In [12]:
#Inserting another image

next(image_loader(data, queue_1))
next(preprocessing(queue_1, queue_2, (300,300)))
next(inference(queue_2, queue_3, 2))

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [13]:
#Number of items in each queue

print("queue 1 size: ", queue_1.qsize())
print("queue 2 size: ", queue_2.qsize())
print("queue 3 size: ", queue_3.qsize())

queue 1 size:  0
queue 2 size:  0
queue 3 size:  1
